In [1]:
from dataclasses import dataclass
from enum import Enum
import random


In [2]:

def gen_tree(
        num_leaves: int, 
        min_num_internals : int = 1,  
        max_num_internals : int = None, 
        min_depth : int = 1, 
        max_depth : int = None, 
        min_degree : int = 2, 
        max_degree : int = None,
        min_leaf_depth: int = 0,
    ):
    """
    Generates a random tree with the given number of leafs, minimum depth, maximum depth, maximum and minimum depth (number of children possible per internal (non-leaf) node), and maximum and minimum number of internal nodes.

    If you need exact number of internal nodes, set min_num_internals = max_num_internals = exact number of internal nodes.

    If you need exact depth, set min_depth = max_depth = exact depth.

    If you need exact degree, set min_degree = max_degree = exact degree.
    """

    if max_degree is None:
        max_degree = num_leaves

    if max_num_internals is None:
        max_num_internals = num_leaves - 1

    if max_depth is None:
        max_depth = float("inf")

    depth_info : dict[int, int] = {}
    parent_info : dict[int, int] = {}
    children_info : dict[int, list[int]] = {}
    
    global_id_iterator = 0
    root_id = global_id_iterator
    children_info[root_id] = []
    depth_info[root_id] = 0
    
    leaves : list[list[int]] = [[root_id]] # depth -> list of leaves of that depth

    current_num_leaves = 1
    current_num_internals = 0

    while current_num_leaves < num_leaves:

        shallow_leaves = {d: l for d, l in enumerate(leaves) if len(l) > 0 and d < min_leaf_depth}
        if len(shallow_leaves) > 0:
            chosen_depth = random.choice(list(shallow_leaves.keys()))

            # Choose a random value for the degree of the parent
            
            current_num_internals += 1
            
            min_tba_num_leaves = sum([len(l) * (min_degree ** (min_leaf_depth - d) - 1) for d, l in shallow_leaves.items()])

            tba_degree_upper_bound_2 = min_degree + (num_leaves - current_num_leaves - min_tba_num_leaves) // (min_degree ** (min_leaf_depth - chosen_depth - 1))

            current_num_leaves -= 1

            tba_degree_lower_bound = max(min_degree, (num_leaves - current_num_leaves) - (max_degree - 1) * max(0, max_num_internals - current_num_internals))

            tba_degree_upper_bound_1 = (num_leaves - current_num_leaves) - (min_degree - 1) * max(0, min_num_internals - current_num_internals)

            tba_degree_upper_bound = min(max_degree, tba_degree_upper_bound_1, tba_degree_upper_bound_2)
            
            if tba_degree_lower_bound > tba_degree_upper_bound:
                raise Exception("No valid tree exists with the given parameters.")
            
            tba_degree = random.randint(tba_degree_lower_bound, tba_degree_upper_bound)

            parent_id_index = random.randint(0, len(shallow_leaves[chosen_depth]) - 1)
            parent_id = shallow_leaves[chosen_depth].pop(parent_id_index)

        elif len(leaves) - 1 < min_depth:
            chosen_depth = len(leaves) - 1
            parent_id_index = random.randint(0, len(leaves[chosen_depth]) - 1)
            parent_id = leaves[chosen_depth].pop(parent_id_index)

            current_num_internals += 1
            current_num_leaves -= 1
            
            # Choose a random value for the degree of the parent
            
            tba_degree_lower_bound = max(min_degree, (num_leaves - current_num_leaves) - (max_degree - 1) * max(0, max_num_internals - current_num_internals))
            
            tba_degree_upper_bound = min(max_degree, (num_leaves - current_num_leaves) - (min_degree - 1) * max(0, min_num_internals - current_num_internals))
            
            tba_degree = random.randint(tba_degree_lower_bound, tba_degree_upper_bound)

        else:
            # Choose a random depth for the parent (the leaf to add children to)
            depths_options = [d for d, v in enumerate(leaves) if len(v) > 0 and d < max_depth]

            if len(depths_options) > 0:
                
                # Choose a random depth for the parent (the leaf to add children to)
                chosen_depth = random.choice(depths_options)
                
                # Choose a random leaf from the current depth
                parent_id_index = random.randint(0, len(leaves[chosen_depth]) - 1)
                parent_id = leaves[chosen_depth].pop(parent_id_index)

                current_num_internals += 1
                current_num_leaves -= 1
                
                # Choose a random value for the degree of the parent
                
                tba_degree_lower_bound = max(min_degree, (num_leaves - current_num_leaves) - (max_degree - 1) * max(0, max_num_internals - current_num_internals))
                
                tba_degree_upper_bound = min(max_degree, (num_leaves - current_num_leaves) - (min_degree - 1) * max(0, min_num_internals - current_num_internals))
                
                tba_degree = random.randint(tba_degree_lower_bound, tba_degree_upper_bound)

            else:
                # Choose a random internal non-full node to add children to.
                non_max_degree_node_options = [_node for _node, _children in children_info.items() if len(_children) < max_degree]
                
                if len(non_max_degree_node_options) == 0:
                    raise Exception("No valid tree exists with the given parameters.")
                    
                # Select a pair of nodes, where the first one is not maximum degree and the second one is not minimum degree, such that the first one is higher (closer to the root) than the second one

                highest_node = min(non_max_degree_node_options, key=lambda _node: depth_info[_node], default=None)

                non_min_degree_node_options = [_node for _node, _children in children_info.items() if len(_children) > min_degree]
                
                deepest_node = max(non_min_degree_node_options, key=lambda _node: depth_info[_node], default=None)

                # If no such pair exists, then the method is not valid. We move to another method: add leafs to existing internal nodes.
                if highest_node is not None and deepest_node is not None and depth_info[highest_node] < depth_info[deepest_node]:
                
                    # Move an edge under the deepest_non_min_degree_node to under the highest_non_max_degree_node
                    
                    # Choose a random child of the deepest_non_min_degree_node
                    child_id = random.choice(children_info[deepest_node])
                    # Remove the child from the children list of the deepest_non_min_degree_node
                    children_info[deepest_node].remove(child_id)
                    # Add the child to the children list of the highest_non_max_degree_node
                    children_info[highest_node].append(child_id)
                    # Update the parent_info
                    parent_info[child_id] = highest_node
                    # Update the depth_info
                    depth_info[child_id] = depth_info[highest_node] + 1
                    # Update depth_info of all descendants of the child, using BFS
                    queue = [child_id]
                    while len(queue) > 0:
                        current_node = queue.pop(0)
                        old_depth = depth_info[current_node]
                        new_depth = depth_info[parent_info[current_node]] + 1
                        if current_node not in children_info:
                            # Update leaves
                            leaves[old_depth].remove(current_node)
                            leaves[new_depth].append(current_node)
                        depth_info[current_node] = new_depth
                        queue += children_info[current_node]
                    continue
                
                # else:
                
                # Add leafs to existing internal nodes.

                parent_id = random.choice(non_max_degree_node_options)

                # Get the depth of the parent
                chosen_depth = depth_info[parent_id]

                # Choose a random value for the to-be-added degree of the parent

                tba_degree_lower_bound = max(1, (num_leaves - current_num_leaves) - (max_degree - 1) * max(0, max_num_internals - current_num_internals))
                
                tba_degree_upper_bound = min(
                    max_degree - len(children_info[parent_id]),
                    (num_leaves - current_num_leaves) - (min_degree - 1) * max(0, min_num_internals - current_num_internals)
                )

                if tba_degree_lower_bound > tba_degree_upper_bound:
                    raise Exception("No valid tree exists with the given parameters.")
                
                tba_degree = random.randint(tba_degree_lower_bound, tba_degree_upper_bound)
            
        if parent_id not in children_info:
            children_info[parent_id] = []
        
        if chosen_depth + 1 >= len(leaves):
            leaves.append([])

        for _ in range(tba_degree):
            global_id_iterator += 1
            leaves[chosen_depth + 1].append(global_id_iterator)
            parent_info[global_id_iterator] = parent_id
            depth_info[global_id_iterator] = chosen_depth + 1
            children_info[parent_id].append(global_id_iterator)
            current_num_leaves += 1

    return depth_info, parent_info, children_info, leaves


In [3]:


class NodeType(Enum):
    AND = "AND"
    OR = "OR"
    LEAF = "LEAF"
    DUMMY = "DUMMY"

def reverse_node_type(node_type):
    if node_type == NodeType.AND:
        return NodeType.OR
    elif node_type == NodeType.OR:
        return NodeType.AND
    else:
        return node_type



def assign_node_type(depth_info: dict[int, int], leaf_nodes : list[int], children_info: dict[int, int], root_node_id : int, strict_and_or: bool = True, root_node_type : NodeType = None):
    """
    Randomly assigns a node type to each node in the tree, where the node type is either "AND", "OR", or "LEAF".
    """
    node_type_info = {}
    if root_node_type is None:
        root_node_type = random.choice([NodeType.AND, NodeType.OR])
    reversed_root_node_type = reverse_node_type(root_node_type)
    node_type_info[root_node_id] = root_node_type
    for node_id in leaf_nodes:
        node_type_info[node_id] = NodeType.LEAF
    for node_id, node_children_ids in children_info.items():
        if node_children_ids is None or len(node_children_ids) == 0:
            node_type_info[node_id] = NodeType.LEAF
            continue

        if node_id == root_node_id:
            continue
        
        if strict_and_or:
            node_depth = depth_info[node_id]
            node_type_info[node_id] = root_node_type if node_depth % 2 == 0 else reversed_root_node_type
        else:
            node_type_info[node_id] = random.choice([NodeType.AND, NodeType.OR])

    return node_type_info


In [ ]:

def traverse_helper(node_id: int, node_type_info: dict, children_info: dict) -> list:
    # If the node is a leaf node (no children)
    if node_type_info[node_id] == NodeType.LEAF:
        yield [node_id]
        return

    # For AND nodes, need to combine children subsets
    if node_type_info[node_id] == NodeType.AND:
        child_subsets = [list(traverse_helper(child, node_type_info, children_info)) for child in children_info[node_id]]
        for combined_subset in combine_subsets_and(child_subsets):
            yield combined_subset
    
    # For OR nodes, simply yield from each child
    elif node_type_info[node_id] == NodeType.OR:
        for child in children_info[node_id]:
            yield from traverse_helper(child, node_type_info, children_info)

def combine_subsets_and(child_subsets: list) -> list:
    # Start with a list containing an empty set
    combined = [[]]
    for subsets in child_subsets:
        new_combined = []
        for subset in subsets:
            for existing in combined:
                new_combined.append(existing + subset)
        combined = new_combined
    return combined

In [46]:
depth_info, parent_info, children_info, leaves = gen_tree(100, max_degree=6)
leaf_nodes = sum(leaves, [])
node_type_info = assign_node_type(depth_info, leaf_nodes, children_info, 0, root_node_type=NodeType.AND)
sorted([(n_id, cv, node_type_info[n_id]) for n_id, cv in children_info.items()], key=lambda x: x[0])

[(0, [1, 2], <NodeType.AND: 'AND'>),
 (1, [9, 10, 11, 12], <NodeType.OR: 'OR'>),
 (2, [3, 4, 5, 6, 7, 8], <NodeType.OR: 'OR'>),
 (3, [64, 65, 66, 67], <NodeType.AND: 'AND'>),
 (4, [42, 43, 44, 45, 46], <NodeType.AND: 'AND'>),
 (5, [27, 28, 29, 30, 31], <NodeType.AND: 'AND'>),
 (6, [17, 18, 19, 20, 21], <NodeType.AND: 'AND'>),
 (7, [80, 81, 82, 83, 84, 85], <NodeType.AND: 'AND'>),
 (8, [13, 14, 15, 16], <NodeType.AND: 'AND'>),
 (9, [22, 23, 24, 25, 26], <NodeType.AND: 'AND'>),
 (10, [47, 48, 49, 50, 51], <NodeType.AND: 'AND'>),
 (11, [38, 39, 40, 41], <NodeType.AND: 'AND'>),
 (12, [68, 69, 70, 71], <NodeType.AND: 'AND'>),
 (19, [32, 33, 34, 35, 36, 37], <NodeType.OR: 'OR'>),
 (30, [61, 62, 63], <NodeType.OR: 'OR'>),
 (32, [52, 53, 54, 55], <NodeType.AND: 'AND'>),
 (40, [130, 131], <NodeType.OR: 'OR'>),
 (47, [115, 116, 117, 118, 119], <NodeType.OR: 'OR'>),
 (49, [56, 57, 58, 59, 60], <NodeType.OR: 'OR'>),
 (52, [72, 73, 74], <NodeType.OR: 'OR'>),
 (54, [120, 121, 122, 123, 124], <NodeTy

In [62]:
def traverse_helper(node_id: int, node_type_info: dict, children_info: dict) -> list:
    
    # print("NODE: ", node_id)

    # If the node is a leaf node (no children)
    if node_type_info[node_id] == NodeType.LEAF:
        # print("YIELD: ", node_id)
        yield [node_id]
        return

    # For AND nodes, need to combine children subsets
    if node_type_info[node_id] == NodeType.AND:
        child_subsets = [list(traverse_helper(child, node_type_info, children_info)) for child in children_info[node_id]]
        for combined_subset in combine_subsets_and(child_subsets):
            # print("YIELD: ", combined_subset)
            yield combined_subset
    
    # For OR nodes, simply yield from each child
    elif node_type_info[node_id] == NodeType.OR:

        # # Randomly choose to skip the node
        # if random.random() < 0.5 and depth_info[node_id] > 2:
        #     print("SKIP: ", node_id)
        #     return

        for child in children_info[node_id]:
            # print("YIELD FROM OR: ", node_id)
            yield from traverse_helper(child, node_type_info, children_info)

def combine_subsets_and(child_subsets: list) -> list:
    # Start with a list containing an empty set
    combined = [[]]
    for subsets in child_subsets:
        new_combined = []
        for subset in subsets:
            for existing in combined:
                new_combined.append(existing + subset)
        combined = new_combined
    return combined




In [50]:
# Traverse the tree and print subsets
for subset in traverse_helper(0, node_type_info=node_type_info, children_info=children_info):
    print(subset)

NODE:  0
NODE:  1
YIELD FROM OR:  1
NODE:  9
NODE:  22
YIELD:  22
NODE:  23
YIELD:  23
NODE:  24
YIELD:  24
NODE:  25
YIELD:  25
NODE:  26
YIELD:  26
YIELD:  [22, 23, 24, 25, 26]
YIELD FROM OR:  1
NODE:  10
NODE:  47
YIELD FROM OR:  47
NODE:  115
YIELD:  115
YIELD FROM OR:  47
NODE:  116
YIELD:  116
YIELD FROM OR:  47
NODE:  117
YIELD:  117
YIELD FROM OR:  47
NODE:  118
YIELD:  118
YIELD FROM OR:  47
NODE:  119
YIELD:  119
NODE:  48
YIELD:  48
NODE:  49
SKIP:  49
NODE:  50
YIELD:  50
NODE:  51
YIELD:  51
YIELD FROM OR:  1
NODE:  11
NODE:  38
YIELD:  38
NODE:  39
YIELD:  39
NODE:  40
SKIP:  40
NODE:  41
YIELD:  41
YIELD FROM OR:  1
NODE:  12
NODE:  68
YIELD:  68
NODE:  69
YIELD:  69
NODE:  70
YIELD:  70
NODE:  71
YIELD:  71
YIELD:  [68, 69, 70, 71]
NODE:  2
YIELD FROM OR:  2
NODE:  3
NODE:  64
YIELD:  64
NODE:  65
YIELD:  65
NODE:  66
YIELD:  66
NODE:  67
YIELD:  67
YIELD:  [64, 65, 66, 67]
YIELD FROM OR:  2
NODE:  4
NODE:  42
YIELD:  42
NODE:  43
YIELD:  43
NODE:  44
YIELD:  44
NODE:  

In [63]:
len(list(traverse_helper(0, node_type_info=node_type_info, children_info=children_info)))

14587

In [70]:
# Traverse the tree and print subsets
list(traverse_helper(0, node_type_info=node_type_info, children_info=children_info))

[[22, 23, 24, 25, 26, 64, 65, 66, 67],
 [115, 48, 56, 50, 51, 64, 65, 66, 67],
 [116, 48, 56, 50, 51, 64, 65, 66, 67],
 [117, 48, 56, 50, 51, 64, 65, 66, 67],
 [118, 48, 56, 50, 51, 64, 65, 66, 67],
 [119, 48, 56, 50, 51, 64, 65, 66, 67],
 [115, 48, 57, 50, 51, 64, 65, 66, 67],
 [116, 48, 57, 50, 51, 64, 65, 66, 67],
 [117, 48, 57, 50, 51, 64, 65, 66, 67],
 [118, 48, 57, 50, 51, 64, 65, 66, 67],
 [119, 48, 57, 50, 51, 64, 65, 66, 67],
 [115, 48, 58, 50, 51, 64, 65, 66, 67],
 [116, 48, 58, 50, 51, 64, 65, 66, 67],
 [117, 48, 58, 50, 51, 64, 65, 66, 67],
 [118, 48, 58, 50, 51, 64, 65, 66, 67],
 [119, 48, 58, 50, 51, 64, 65, 66, 67],
 [115, 48, 59, 50, 51, 64, 65, 66, 67],
 [116, 48, 59, 50, 51, 64, 65, 66, 67],
 [117, 48, 59, 50, 51, 64, 65, 66, 67],
 [118, 48, 59, 50, 51, 64, 65, 66, 67],
 [119, 48, 59, 50, 51, 64, 65, 66, 67],
 [115, 48, 60, 50, 51, 64, 65, 66, 67],
 [116, 48, 60, 50, 51, 64, 65, 66, 67],
 [117, 48, 60, 50, 51, 64, 65, 66, 67],
 [118, 48, 60, 50, 51, 64, 65, 66, 67],
 

In [73]:
def traverse_and_or_tree_from_leaves_subset(leaves_subset: list, parent_info: dict, node_type_info: dict, children_info: dict):
    # Function to find the parent nodes of the leaves in the starting subset
    def find_parent_nodes(leaves):
        return set(parent_info[leaf] for leaf in leaves if leaf in parent_info)

    # Recursive function to continue traversal
    def continue_traversal(node_id: int, visited: set):
        if node_id in visited or node_id not in children_info:
            return

        if node_type_info[node_id] == NodeType.LEAF:  # Leaf node
            # visited.add(node_id)
            if node_id in leaves_subset:
                yield [node_id]
            return

        if node_type_info[node_id] == NodeType.AND:
            # visited.add(node_id)
            child_subsets = [list(continue_traversal(child, visited.copy())) for child in children_info[node_id]]
            for combined_subset in combine_subsets_and(child_subsets):
                yield combined_subset
                
        elif node_type_info[node_id] == NodeType.OR:
            for child in children_info[node_id]:
                yield from continue_traversal(child, visited.copy())

    # Start the traversal from each parent node of the leaves in the subset
    parent_nodes = find_parent_nodes(leaves_subset)
    for parent_node in parent_nodes:
        yield from continue_traversal(parent_node, set())

def combine_subsets_and(child_subsets: list[list[list]]) -> list:
    combined = [[]]
    for subsets in child_subsets:
        new_combined = []
        for subset in subsets:
            for existing in combined:
                new_combined.append(existing + subset)
        combined = new_combined
    return combined

In [75]:
leaves_subset = [68, 69, 70, 71, 42, 43, 44, 45, 46]
list(traverse_and_or_tree_from_leaves_subset(leaves_subset, parent_info, node_type_info, children_info))

[]

In [251]:
def traverse_and_or_tree(node_type_info: dict, children_info: dict, depth_info: dict, root_node_id: int = 0):
    """
    Traverses an AND-OR goal tree and yields all possible solutions (subsets of leaves that can be completed to fulfill an AND-OR goal tree).
    """

    # skipped_nodes = set()

    def traverse_helper(node_id: int) -> list:
        
        # print("NODE: ", node_id)

        # if node_type_info[node_id] != NodeType.OR:
        #     if random.random() < 0.1 and depth_info[node_id] > 2:
        #         skipped_nodes.add(node_id)
        #         return

        # If the node is a leaf node (no children)
        if node_type_info[node_id] == NodeType.LEAF:
            # print("YIELD: ", node_id)
            yield [node_id]
            return

        # For AND nodes, need to combine children subsets
        if node_type_info[node_id] == NodeType.AND:
            
            # leaves_subsets = [list(traverse_and_or_tree(child)) for child in children_info[node_id]]

            stack = [([], 0)]
            
            while stack:
                combination, index = stack.pop()
                if index >= len(children_info[node_id]):
                    yield combination
                else:
                    # for item in leaves_subsets[index]:
                    for item in traverse_helper(children_info[node_id][index]):
                        stack.append((combination + [item], index + 1))

        
        # For OR nodes, simply yield from each child
        elif node_type_info[node_id] == NodeType.OR:

            # num_child = len(children_info[node_id])
            
            for child in children_info[node_id]:
                # if random.random() < 0.1 and depth_info[child] > 2 and num_child > 1:
                #     num_child -= 1
                #     skipped_nodes.add(child)
                #     continue
                yield from traverse_helper(child)

    yield from traverse_helper(root_node_id)

    # return list(traverse_helper(root_node_id)), skipped_nodes



In [255]:
[list(range(0, 10))]

[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]]

In [241]:
def backtrack_combinations(lists, path=[]):
    if not lists:
        yield path
    else:
        for item in lists[0]:
            yield from backtrack_combinations(lists[1:], path + [item])

# Example usage
lists = [[1, 2], [3, 4], [5, 6]]
for combination in backtrack_combinations(lists):
    print(combination)


[1, 3, 5]
[1, 3, 6]
[1, 4, 5]
[1, 4, 6]
[2, 3, 5]
[2, 3, 6]
[2, 4, 5]
[2, 4, 6]


In [254]:
t, s = traverse_and_or_tree(node_type_info, depth_info, children_info)
len(t), len(s)

(14587, 0)

In [84]:
combine_subsets_and([[[1, 2], [3, 4]], [[5, 6], [7, 8]]])

[[1, 2, 5, 6], [3, 4, 5, 6], [1, 2, 7, 8], [3, 4, 7, 8]]